In [1]:
!pip install wandb

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from itertools import product
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from copy import deepcopy
import time
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import wandb
wandb.login(key='0057f517a0acb0992a4695e1d89cff691b8a8960')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fraschetti-1834942. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Using device: cuda


In [3]:
SABR_price = pd.read_csv('/kaggle/input/sabr-def-p1/SABR_model_DEF_PARTE01.csv')
SABR_price["S0"] = SABR_price["S0"] * np.exp(SABR_price["r"] * SABR_price["T"])
SABR_price.head()

,S0,alpha,beta,rho,nu,r,T,K,IV,IV_Approx,BS Price,SABR Price,BS Price IV
0,85.920476,0.067447,1.0,0.520296,0.124547,0.024805,0.2,50,0.386060,0.060740,35.742719,35.745352,35.745352
1,85.920476,0.067447,1.0,0.520296,0.124547,0.024805,0.2,55,0.321857,0.060176,30.767462,30.770095,30.770095
2,85.920476,0.067447,1.0,0.520296,0.124547,0.024805,0.2,60,0.262809,0.060290,25.792205,25.794838,25.794838
3,85.920476,0.067447,1.0,0.520296,0.124547,0.024805,0.2,65,0.207931,0.060993,20.816948,20.819582,20.819582
4,85.920476,0.067447,1.0,0.520296,0.124547,0.024805,0.2,70,0.156389,0.062161,15.841691,15.844325,15.844325


In [5]:
class OptionPricingDatasetSABR(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32).to(device)
        self.y = torch.tensor(y.values, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [10]:
X_sabr_vol = SABR_price.drop(['IV_Approx','IV', 'BS Price IV'], axis=1)  # Features
y_sabr_vol = SABR_price['IV']            # Target

X_train_sabr_vol , X_temp_sabr_vol , y_train_sabr_vol , y_temp_sabr_vol  = train_test_split(X_sabr_vol , y_sabr_vol , test_size=0.2, random_state=42)         # Train/Test+Val
X_val_sabr_vol , X_test_sabr_vol , y_val_sabr_vol , y_test_sabr_vol  = train_test_split(X_temp_sabr_vol, y_temp_sabr_vol, test_size=0.5, random_state=42)  # Val/Test


scaler = MinMaxScaler()
X_train_normalized_sabr_vol  = scaler.fit_transform(X_train_sabr_vol)  # Fit on training data and transform
X_val_normalized_sabr_vol  = scaler.transform(X_val_sabr_vol)         # Transform validation data
X_test_normalized_sabr_vol  = scaler.transform(X_test_sabr_vol)       # Transform test data

# Convert normalized data back into DataFrames for DataLoader compatibility
X_train_normalized_sabr_vol  = pd.DataFrame(X_train_normalized_sabr_vol , columns=X_sabr_vol.columns)
X_val_normalized_sabr_vol  = pd.DataFrame(X_val_normalized_sabr_vol , columns=X_sabr_vol.columns)
X_test_normalized_sabr_vol  = pd.DataFrame(X_test_normalized_sabr_vol , columns=X_sabr_vol.columns)

# Create PyTorch datasets
train_dataset_sabr_vol  = OptionPricingDatasetSABR(X_train_normalized_sabr_vol , y_train_sabr_vol)
val_dataset_sabr_vol  = OptionPricingDatasetSABR(X_val_normalized_sabr_vol , y_val_sabr_vol)
test_dataset_sabr_vol  = OptionPricingDatasetSABR(X_test_normalized_sabr_vol , y_test_sabr_vol)

# Create DataLoaders
#train_loader_sabr_cal  = torch.utils.data.DataLoader(train_dataset_sabr_cal , batch_size=64, shuffle=True)
#val_loader_sabr_cal  = torch.utils.data.DataLoader(val_dataset_sabr_cal , batch_size=64, shuffle=False)
#test_loader_sabr_cal  = torch.utils.data.DataLoader(test_dataset_sabr_cal , batch_size=64, shuffle=False)

In [19]:
def train_model_sabr(model, train_loader, val_loader, mse_criterion, optimizer, device, num_epochs=50, patience=5):
    best_loss = float('inf')
    counter = 0
    best_model = deepcopy(model.state_dict())
    all_predictions = []
    all_targets = []
    for epoch in range(num_epochs):
        model.train()
        running_loss_mse = 0.0
        for inputs, targets in tqdm(train_loader, desc="Training Epoch", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            optimizer.zero_grad()
            outputs = model(inputs)
            targets = targets.unsqueeze(1)
            
            # Compute MSE loss for monitoring (not used in training)
            
            loss_mse = mse_criterion(outputs, targets)
            
            # Backward pass and optimization
            loss_mse.backward()
            optimizer.step()

            running_loss_mse += loss_mse.item()
           
            # Collect predictions and true values for MAE & R² computation
            all_predictions.extend(outputs.cpu().detach().numpy().flatten())
            all_targets.extend(targets.cpu().detach().numpy().flatten())
            
        avg_train_loss_mse = running_loss_mse / X_train_sabr_vol.shape[0]
        avg_train_loss_mae = mean_absolute_error(all_targets, all_predictions)
        train_r2_score = r2_score(all_targets, all_predictions)


        
        print(f"Epoch {epoch + 1}/{num_epochs} - "
              f"Train MSE: {avg_train_loss_mse:.10f}, "
              f"Train MAE: {avg_train_loss_mae:.10f}, "
              f"Train R²: {train_r2_score:.10f}")

        #print(f"Epoch {epoch + 1}/{num_epochs},  Train MSE Loss: {avg_train_loss_mse:.10f}")

        # Validation after each epoch
        avg_val_loss_mse = validation_model_sabr(model, val_loader, mse_criterion, device)


        # Early stopping based on custom loss
        if avg_val_loss_mse < best_loss:
            best_loss = avg_val_loss_mse
            counter = 0
            best_model = deepcopy(model.state_dict())  # Save the best model state
        else:
            counter += 1

        if counter >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    model.load_state_dict(best_model)  # Restore the best model
    return model, avg_train_loss_mse, avg_val_loss_mse, best_loss

# Validation function
def validation_model_sabr(model, val_loader,  mse_criterion, device):
    model.eval()
    val_loss_mse = 0.0
    all_predictions = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in tqdm(val_loader, desc="Validation", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            targets = targets.unsqueeze(1)

            
            loss_mse = mse_criterion(outputs, targets)

            val_loss_mse += loss_mse.item()
            
            # Collect predictions and true values for MAE & R² computation
            all_predictions.extend(outputs.cpu().detach().numpy().flatten())
            all_targets.extend(targets.cpu().detach().numpy().flatten())
            
        avg_val_loss_mse = val_loss_mse / X_val_sabr_vol.shape[0]
        avg_val_loss_mae = mean_absolute_error(all_targets, all_predictions)
        val_r2_score = r2_score(all_targets, all_predictions)

        print(f"Val MSE: {avg_val_loss_mse:.10f}, "
              f"Val  MAE: {avg_val_loss_mae:.10f}, "
              f"Val  R²: {val_r2_score:.10f}")
    #print(f" Validation MSE Loss: {avg_val_loss_mse:.10f}")
    return  avg_val_loss_mse

# Testing function
def test_model_sabr(model, test_loader, mse_criterion, device):
    model.eval()
    
    test_loss_mse = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in tqdm(test_loader, desc="Testing", unit="batch"):
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            targets = targets.unsqueeze(1)

            loss_mse = mse_criterion(outputs, targets)        # MSE loss

            test_loss_mse += loss_mse.item()

            all_predictions.extend(outputs.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    avg_test_loss_mse = test_loss_mse / X_test_sabr_vol.shape[0]
    avg_test_loss_mae = mean_absolute_error(all_targets, all_predictions)
    test_r2_score = r2_score(all_targets, all_predictions)

    print(f"Test MSE: {avg_test_val_loss_mse:.10f}, "
          f"Test MAE: {avg_test_val_loss_mae:.10f}, "
          f"Test R²: {test_r2_score:.10f}")

    return  avg_test_loss_mse, predictions, targets_list

In [20]:
def objective(trial):
    # Define hyperparameters to optimize
    num_layers = trial.suggest_int("num_layers", 2, 5)  # Number of hidden layers
    hidden_size = trial.suggest_int("hidden_size", 50, 500)  # Number of neurons per layer
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)  # Dropout rate
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-3, log=True)  # Learning rate
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])  # Batch size

    wandb.init(project="SABR-Optimization_vol", name=f"trial-{trial.number}", mode="online")

    # Define the MLP model dynamically
    class MLP(nn.Module):
        def __init__(self, input_dim, num_layers, hidden_size, dropout_rate):
            super(MLP, self).__init__()
            layers = [nn.Linear(input_dim, hidden_size), nn.ReLU(), nn.Dropout(dropout_rate)]
            for _ in range(num_layers - 1):
                layers.extend([nn.Linear(hidden_size, hidden_size), nn.ReLU(), nn.Dropout(dropout_rate)])
            layers.extend([nn.Linear(hidden_size, int(hidden_size/4)), nn.ReLU(), nn.Dropout(dropout_rate)])
            layers.extend([nn.Linear(int(hidden_size/4), int(hidden_size/4)), nn.ReLU(), nn.Dropout(dropout_rate)])
            layers.append(nn.Linear(int(hidden_size/4), 1))  # Output layer
            self.model = nn.Sequential(*layers)

        def forward(self, x):
            return self.model(x)

    # Initialize model
    input_dim = X_train_sabr_cal.shape[1]
    model = MLP(input_dim, num_layers, hidden_size, dropout_rate).to(device)

    # Define loss function and optimizer
    mse_criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Create DataLoaders with the suggested batch_size
    train_loader_sabr = torch.utils.data.DataLoader(train_dataset_sabr_vol, batch_size=batch_size, shuffle=True)
    val_loader_sabr = torch.utils.data.DataLoader(val_dataset_sabr_vol, batch_size=batch_size, shuffle=False)

    # Train the model
    trained_model, _, _, best_val_loss = train_model_sabr(
        model, train_loader_sabr, val_loader_sabr, mse_criterion, optimizer, device, num_epochs=50, patience=5
    )

    # Log hyperparameters and loss to W&B
    wandb.log({
        "num_layers": num_layers,
        "hidden_size": hidden_size,
        "dropout_rate": dropout_rate,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "val_loss": best_val_loss
    })

    wandb.finish()
    
    return best_val_loss  # Minimize validation loss

# Run Optuna study with the updated objective function
study = optuna.create_study(study_name="MLP_Optimization_vol", direction="minimize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)  # Try 30 different configurations
# Log the best results
wandb.init(project="SABR-Optimization_vol", name="Best_Trial")
wandb.log(study.best_params)
wandb.finish()
# Print best hyperparameters
print("Best Hyperparameters:", study.best_params)

[I 2025-03-10 00:19:55,044] A new study created in memory with name: MLP_Optimization_vol
Training Epoch: 100%|██████████| 48765/48765 [02:06<00:00, 386.23batch/s]


Epoch 1/50 - Train MSE: 0.0000070140, Train MAE: 0.0107576968, Train R²: 0.9094055954


Validation: 100%|██████████| 6096/6096 [00:06<00:00, 949.88batch/s]


Val MSE: 0.0000032798, Val  MAE: 0.0071503120, Val  R²: 0.9575533212


Training Epoch: 100%|██████████| 48765/48765 [02:06<00:00, 385.57batch/s]


...

Val MSE: 0.0000077695, Val  MAE: 0.0157265384, Val  R²: 0.8994461423
Early stopping triggered at epoch 6


batch_size,▁
dropout_rate,▁
hidden_size,▁
learning_rate,▁
num_layers,▁
val_loss,▁
batch_size,64
dropout_rate,0.09592
hidden_size,102
learning_rate,0.00056
num_layers,5


[I 2025-03-10 11:38:39,955] Trial 29 finished with value: 4.972641728787203e-06 and parameters: {'num_layers': 5, 'hidden_size': 102, 'dropout_rate': 0.09592026181315833, 'learning_rate': 0.0005609807690808686, 'batch_size': 64}. Best is trial 12 with value: 2.1761406455312922e-07.


batch_size,▁
dropout_rate,▁
hidden_size,▁
learning_rate,▁
num_layers,▁
batch_size,256
dropout_rate,0.00218
hidden_size,51
learning_rate,0.00037
num_layers,5


Best Hyperparameters: {'num_layers': 5, 'hidden_size': 51, 'dropout_rate': 0.002176982402411834, 'learning_rate': 0.00036716424635711327, 'batch_size': 256}
